In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
df_list = []

# 데이터프레임 생성
for dataset in os.listdir('../dataset'):
    if dataset.endswith('.csv'):
        globals()[dataset[:-4]] = pd.read_csv(f'../dataset/{dataset}')
        print(dataset[:-4])
        df_list.append(dataset[:-4])

products
orders
order_products__train
departments
aisles
order_products__prior
sample_submission


In [4]:
# train : val : test => prior : train : test
# for all user's latent factors
train_data = orders[orders['eval_set']=='prior']
val_data = orders[orders['eval_set']=='train']
test_data = orders[orders['eval_set']=='test']

In [5]:
# ratio of each dataset
print(len(train_data) / (len(train_data) + len(val_data) + len(test_data)))
print(len(val_data) / (len(train_data) + len(val_data) + len(test_data)))
print(len(test_data) / (len(train_data) + len(val_data) + len(test_data)))

0.9397240581418224
0.03835305954284067
0.021922882315336987


In [5]:
def preprocessing(data, order_products):
    user_id = data[['order_id', 'user_id']]
    data = data.drop(columns=['user_id', 'eval_set'])
    data = data.groupby('order_id').agg('mean').sort_values('order_id').reset_index()
    data = data.merge(user_id, on='order_id')
    merged_data = pd.merge(order_products[['order_id', 'reordered']], data, on='order_id')
    return merged_data

In [10]:
orders_subset = orders[['order_id', 'user_id']]
order_products_prior_with_user = pd.merge(order_products__prior, orders_subset, on='order_id', how='left')

# 결과 출력
order_products_prior_with_user.head()

,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,9327,3,0,202279
3,2,45918,4,1,202279
4,2,30035,5,0,202279


In [13]:
user_product_list = order_products_prior_with_user.groupby('order_id')['product_id'].apply(list).reset_index()
user_product_list

,order_id,product_id
0,2,"[33120, 28985, 9327, 45918, 30035, 17794, 4014..."
1,3,"[33754, 24838, 17704, 21903, 17668, 46667, 174..."
2,4,"[46842, 26434, 39758, 27761, 10054, 21351, 225..."
3,5,"[13176, 15005, 47329, 27966, 23909, 48370, 132..."
4,6,"[40462, 15873, 41897]"
...,...,...
3214869,3421079,[30136]
3214870,3421080,"[27845, 4932, 18811, 41950, 31717, 12935, 2512..."
3214871,3421081,"[38185, 12218, 32299, 3060, 20539, 35221, 12861]"
3214872,3421082,"[17279, 12738, 16797, 43352, 32700, 12023, 47941]"


In [12]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [6]:
train = preprocessing(train_data, order_products__prior)
val = preprocessing(val_data, order_products__train)

In [7]:
train

,order_id,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id
0,2,1,3.0,5.0,9.0,8.0,202279
1,2,1,3.0,5.0,9.0,8.0,202279
2,2,0,3.0,5.0,9.0,8.0,202279
3,2,1,3.0,5.0,9.0,8.0,202279
4,2,0,3.0,5.0,9.0,8.0,202279
...,...,...,...,...,...,...,...
32434484,3421083,1,24.0,2.0,6.0,21.0,25247
32434485,3421083,0,24.0,2.0,6.0,21.0,25247
32434486,3421083,0,24.0,2.0,6.0,21.0,25247
32434487,3421083,1,24.0,2.0,6.0,21.0,25247


In [81]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분할
X_train, y_train = train.drop(columns='reordered'), train['reordered']
X_test, y_test = val.drop(columns='reordered'), val['reordered']

# XGBoost 모델 정의
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# 모델 학습
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(classification_report(y_test, y_pred))

Accuracy: 0.6335983163575198
F1 Score: 0.7420624590717367
              precision    recall  f1-score   support

           0       0.60      0.27      0.37    555793
           1       0.64      0.88      0.74    828824

    accuracy                           0.63   1384617
   macro avg       0.62      0.57      0.55   1384617
weighted avg       0.62      0.63      0.59   1384617



In [84]:
X_test

,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id
0,1,4.0,4.0,10.0,9.0,112108
1,1,4.0,4.0,10.0,9.0,112108
2,1,4.0,4.0,10.0,9.0,112108
3,1,4.0,4.0,10.0,9.0,112108
4,1,4.0,4.0,10.0,9.0,112108
...,...,...,...,...,...,...
1384612,3421063,30.0,0.0,10.0,4.0,169679
1384613,3421063,30.0,0.0,10.0,4.0,169679
1384614,3421070,15.0,6.0,10.0,8.0,139822
1384615,3421070,15.0,6.0,10.0,8.0,139822


In [82]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [143]:
test_data = orders[orders['eval_set']=='test']
features = test_data[['order_id', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']]
v_pred = model.predict(features)

In [147]:
test_data

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0
...,...,...,...,...,...,...,...
3420918,2728930,206202,test,23,2,17,6.0
3420929,350108,206204,test,5,4,14,14.0
3421001,1043943,206206,test,68,0,20,0.0
3421018,2821651,206207,test,17,2,13,14.0


In [111]:
o_id = order_products__train.groupby('order_id').apply(lambda x: ' '.join(map(str, x['product_id']))).reset_index()
p_ids = merged_data[v_pred == 1][['order_id', 'product_id']].sort_values('order_id').groupby('order_id').apply(lambda x: ' '.join(map(str, x['product_id']))).reset_index()
o_id.columns = ['order_id', 'p']
p_ids.columns = ['order_id', 'products']

In [122]:
result = o_id.merge(p_ids, how='outer', on='order_id').iloc[:, [0, 2]]

In [134]:
result.set_index('order_id').to_csv('/Users/geonmin/result.csv')

In [148]:
order_products__prior

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [154]:
orders[orders['order_id']==3421083]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
419472,3421083,25247,prior,24,2,6,21.0
